# Stoneburner, Kurt
- ## DSC 550 - Week 09/10

Due the to complex nature of this assignment I've broken the assignment into three notebooks. Assignment 3 would only run on Intel PCs I have an AMD processor. I managed to address this issue by connectly remotely to a work PC and doing the assignment there. This first assignment was a real resource hog. I tried to apply some learnings from Keras to improve the model accurracy but I simply didn't have the time for proper trial and error. 

In [1]:
import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json



#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

### 1. Neural Network Classifier with Scikit ### 

Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using scikit-learn. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.


In [2]:
#//***************************************
#//*** Apply Common Cleanup operations
#//***************************************
#//*** These cleanup functions are based on Week 02 cleanup code, and rebuilt for Week 04

#//*****************************************
#//*** Functions:
#//*****************************************
#//*** Mr_clean_text: Converts to lowercase, removes punctuation, newlines and html markup
#//****************************************************************************************************
#//*** Tokenize_series: Converts a Series containing strings, to a series containing tokenized lists
#//****************************************************************************************************
#//*** Remove_stop_words: Removes Stop words based on nltk stopwords 'english' dictionary
#//****************************************************************************************************
#//*** Apply_stemmer: Stem tokenized words using nltk.stem.porter.PorterStemme
#//****************************************************************************************************
#//*** apply_pos_tag: Builds Part of Speech Tagging from tokeninzed text
#//****************************************************************************************************

#//****************************************************************************************************

#//****************************************************************************************************
#//*** Key values will default to true. If code needs to be defaulted to False, a default_false list can be added later
#//*** All Boolean kwarg keya are stored in kwarg list. This speeds up the coding of the action_dict.
#//*** As Kwargs are added 
def mr_clean_text(input_series, input_options={}):
    
    def clean_text(input_string):
        clean1 = re.sub(r'['+string.punctuation + '’—”'+']', "", input_string.lower())
        return re.sub(r'\W+', ' ', clean1)

    #//*** import time library
    try:
        type(time)
    except:
        import time
    
    #//*** Force creation of a new Series instead of using a copy
    input_series = input_series.copy()
    
    #//*** Start Timing the process
    start_time = time.time()

    
    #//*** Add some data validation. I'm preparing this function for additional use. I'm checking if future users (ie future me)
    #//*** may throw some garbage at this function. Experience has taught me to fail safely wherever possible.

    #//*** All kwargs are listed here. These initialize TRUE by default.
    key_list = [ "lower", "newline", "html", "punctuation" ]
    
    default_false = ["remove_empty"]
    
    #//*** Build Action Dictionary
    action_dict = { } 
    
    #//*** Build the keys from kwarg_list and default them to TRUE
    for key in key_list:
        action_dict[key] = True
    
    for key in default_false:
        action_dict[key] = False
        
    #//*** Loop through the input kwargs (if any). Assign the action_dict values based on the kwargs:
    for key,value in input_options.items():
        print(key,value)
        action_dict[key] = value
    
    
    #//*************************************************************************
    #//*** The Cleanup/Processing code is a straight lift from DSC550 - Week02
    #//*************************************************************************
    #//*** Convert to Lower Case, Default to True
    if action_dict["lower"]:
        input_series = input_series.str.lower()
    
   
    #//*** Remove New Lines
    if action_dict["newline"]:
        #//*** Rmove \r\n
        input_series = input_series.str.replace('\r?\n',"")

        #//*** Remove \n new lines
        input_series = input_series.str.replace('\n',"")
    
    
    input_series = input_series.str.replace("\\(http.+\\)","")
    
    #//*** Print Elements between brackets
    #print(input_series[ input_series == input_series.str.match('[.*]')])

     
    #//*** Remove html entities, observed entities are &gt; and &lt;. All HTML entities begin with & and end with ;.
    #//*** Let's use regex to remove html entities
    if action_dict["html"]:
        input_series = input_series.str.replace(r'&.*;',"")

    #//*** Remove the empty lines
    if action_dict["remove_empty"]:
        input_series = input_series[ input_series.str.len() > 0]

    #//*** Remove punctuation
    if action_dict["punctuation"]:
        #//*** Load libraries for punctuation if not already loaded.
        #//*** Wrapping these in a try, no sense in importing libraries that already exist.
        #//*** Unsure of the cost of reimporting libraries (if any). But testing if library is already loaded feels
        #//*** like a good practice

        #input_series = input_series.apply(lambda x: clean_text(x))

        try:
            type(sys)
        except:
            import sys

        try:
            type(unicodedata)
        except:
            import unicodedata

        #//*** replace Comma and Period with a space.
        for punct in [",","."]:
            input_series = input_series.str.replace(punct," ")

        #//*** Remove punctuation using the example from the book
        punctuation = dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P') )
        input_series = input_series.str.translate(punctuation)

        #table = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}
        #print(table )
        #input_series = input_series.str.translate(table)

    print(f"Text Cleaning Time: {time.time() - start_time}")

    return input_series

                                          
#//*** Tokenize a Series containing Strings.
#//*** Breaking this out into it's own function for later reuse.
#//*** Not a lot of code here, but it helps to keep the libraries localized. This creates standarization for future
#//*** Stoneburner projects. Also has the ability to add functionality as needed.

def tokenize_series(input_series,slices=20,input_options={}):
    
    try:
        type(nltk)
    except:
        import nltk
    
    word_tokenize = nltk.tokenize.word_tokenize 
    
    #//*** import time library
    try:
        type(time)
    except:
        import time
        
    #//*** All kwargs are listed here. These initialize False by default.
    key_list = [ "fast", "quiet" ]
    
    #//*** Build Action Dictionary
    action_dict = { } 
    
    #//*** Build the keys from kwarg_list and default them to False
    for key in key_list:
        action_dict[key] = False
        
    #//*** Loop through the input kwargs (if any). Assign the action_dict values based on the kwargs:
    for key,value in input_options.items():
        print(key,value)
        action_dict[key] = value
    
    #//*** Start Timing the process
    start_time = time.time()
    
    #//*** Force creation of a new Series instead of using a copy
    input_series = input_series.copy()
            
    #input_series = input_series.apply(word_tokenize)
    
    if action_dict['fast'] == False:
        print("Processing Tokens with NLTK Word Tokenize")
        input_series = apply_with_progress(input_series,word_tokenize,slices)
    else:
        print("Process Tokens with Split()")
        input_series = apply_with_progress(input_series,lambda x: x.split(),slices)
    
    
    
    print(f"Tokenize Time: {time.time() - start_time}")
    
    return input_series

#//*** Remove Stop words from the input list
def remove_stop_words(input_series):
    
    #//*** This function removes stop_words from a series.
    #//*** Works with series.apply()
    def apply_stop_words(input_list):

        #//*** Load Stopwords   
        for word in input_list:
            if word in stop_words:
                input_list.remove(word)
                #print(f"Removing: {word}")
        return input_list

    #//*** import nltk if needed
    try:
        type(nltk)
    except:
        import nltk
        
    stopwords = nltk.corpus.stopwords

    #//*** Stopwords requires an additional download
    try:
        type(stopwords)
    except:
        nltk.download('stopwords')


    #//*** import time library
    try:
        type(time)
    except:
        import time

    #//*** Start Timing the process
    start_time = time.time()

    #//*** Force creation of a new Series instead of using a copy
    input_series = input_series.copy()
    
    #//*** The stop_words include punctuation. Stop Word Contractions will not be filtered out.
    #//*** Manually adding word the
    stop_words = []
    
    #//*** Remove apostrophies from the stop_words
    for stop in stopwords.words('english'):
        stop_words.append(stop.replace("'",""))

    #print("Stop Words: ")
    print(stop_words)
    print ("Processing Stop Words")
    input_series = apply_with_progress(input_series, apply_stop_words)
    
    print(f"Stop Words Time: {time.time() - start_time}")
    
    return input_series

def apply_stemmer(input_series,trim_single_words = True,slices=100):
    #//*** import nltk if needed
    try:
        type(nltk)
    except:
        import nltk

    #//*** Instantiate the Stemmer
    porter = nltk.stem.porter.PorterStemmer()
    
    #//*** import time library
    try:
        type(time)
    except:
        import time

    #//*** Start Timing the process
    start_time = time.time()
    
    #//*** Force creation of a new Series instead of using a copy
    input_series = input_series.copy()
    
    #//*** 1.) Apply() an action to each row
    #//*** 2.) lambda word_list, each row is treated as word_list for the subsequent expression
    #//*** 3.) The base [ word for word in wordlist] would return each word in word_list as a list. 
    #//*** 4.) [porter.stem(word) for word in word_list] - performs stemming on each word and returns a list
    #input_series = input_series.apply(lambda word_list: [porter.stem(word) for word in word_list] )
    print("Begin: Apply Stemmer")
    input_series = apply_with_progress(input_series, lambda word_list: [porter.stem(word) for word in word_list],slices)
    
    #//*** Remove Single letter words after stemming
    
    """
    if trim_single_words:
        for word_list in input_series:
            for word in word_list:
                if len(word) < 2:
                    word_list.remove(word)
    """
    
    print(f"Apply Stemmer Time: {time.time() - start_time}")
    return input_series

def apply_pos_tag(input_series,slices=100):
    #//*** import nltk if needed
    try:
        type(nltk)
    except:
        import nltk

    from nltk import pos_tag

    #//pos_tag requires an additional download
    try:
        pos_tag(["the","quick","brown","fox"])
    except: 
        nltk.download('averaged_perceptron_tagger')
    
    #//*** import time library
    try:
        type(time)
    except:
        import time

    #//*** Start Timing the process
    start_time = time.time()
    
    #//*** Force creation of a new Series instead of using a copy
    input_series = input_series.copy()
    
    print("Begin Part of Speech tagging")
    
    input_series = apply_with_progress(input_series,pos_tag,slices)
    
    print(f"Part of Speech Tagging Time: {round(time.time() - start_time,2)}s")
    
    return input_series
    
def apply_lemmatization(input_series,slices=20):
            
    #//*** import nltk if needed
    try:
        type(nltk)
    except:
        import nltk


    from nltk.stem import WordNetLemmatizer
    
    from nltk.corpus import wordnet    
    
    #nltk.download('wordnet')
    
    
    #//*** import time library
    try:
        type(time)
    except:
        import time

    #//*** Start Timing the process
    start_time = time.time()
    
    #//*** Force creation of a new Series instead of using a copy
    input_series = input_series.copy()
    
    # Initialize the Lemmatizer instance
    lemmatizer = WordNetLemmatizer() 
    
    #//*** 1.) Apply() an action to each row
    #//*** 2.) lambda word_list, each row is treated as word_list for the subsequent expression
    #//*** 3.) The base [ word for word in wordlist] would return each word in word_list as a list. 
    #//*** 4.) [lemmatizer.lemmatize(word) for word in word_list] - performs lemmtization on each word and returns a list
    #lemmatized = input_series.apply(lambda word_list: [lemmatizer.lemmatize(*word) for word in word_list] )
    
    print("Begin Lemmatization...")
    
    input_series = apply_with_progress(input_series,lambda word_list: [lemmatizer.lemmatize(word) for word in word_list],20)
    
    print(f"Lemmatization Time: {time.time() - start_time}")
    
    #if detoken:
    #    return tokenize_series(input_series,5,{"fast":True})

    return input_series

#//*** Apply a function to a Series and display processing progress
#//*** Slices is the total number of intervals to report progress.
#//*** Slices = 20 displays processing after every 5% is processed
#//*** Slices = 100 displays processing after every 1% is processed
def apply_with_progress(input_series,input_function,slices=20):
    #//*** import time library
    try:
        type(time)
    except:
        import time

    #//*** Get the time at the start of the loop, used for elapsed time.
    start_time = time.time()
    
    #//*** The interval is the number of elements to process in each Loop. The default is 20.
    #//*** Which displays results at 5% intervals.
    interval = int(len(input_series)/slices)
    
    #//*** Total number of items to process
    total = len(input_series)
    

    #//*** Loop through slice times and display processing statistics for each slice.
    for x in range(0, slices ):
        #//*** Get time at the start of the slice.
        loop_time = time.time()
        
        #//*** Set the start index
        begin_dex = interval*x
        
        #//*** Set the end index
        end_dex = interval*x+interval-1
        
        #//*** Apply the input function to a slice of the input_series
        #//*** This part does all the actual 'work'
        input_series[begin_dex:end_dex] = input_series[begin_dex:end_dex].apply(input_function)
        
        #//*** Get the time after the slice of work is done
        now = time.time()
        
        #//*** Compute the estimated remaining time
        #//*** Total elapsed time / % of completed slices = Estimated total time
        #//*** Estimated total time - elaped time = Remaining time
        est_remain = round( ( ( now - start_time ) /  ( (x+1)/slices ) - (now-start_time)),2)

        #//*** Display Results so we know how much time is left (so we can effectively multi-task: ie comments, research and Valheim)
        print(f"Processed {x}/{slices}: {begin_dex}:{end_dex} [{total}] in {round(now-loop_time,2)}s elapsed: {round(now-start_time,2)}s est Remain: {est_remain}s")
    
    #//*** END For Slice Loop
    
    #//*** Process the remaining values (Since interval is an int there should be a remainder)
    loop_time = time.time()
    begin_dex = end_dex+1
    if begin_dex < len(input_series):
        print(f"Processing Remaining values: {begin_dex} : {total} ")
        #print(input_series[begin_dex:])
        input_series[begin_dex:] = input_series[begin_dex:].apply(input_function)
    
    #//*** Display Final output
    print(f"Processed {slices}/{slices}: {begin_dex}:{end_dex} [{total}] in {round(time.time()-loop_time,2)}s elapsed: {round(time.time()-start_time,2)}s")
    
    #//*** return Series
    return input_series

In [3]:
#//******************************************************************
#//*** Process categorized-comments.jsonl and create a local copy
#//******************************************************************
#//*** Processed comments are stored in a local pickled file. 
#//*** Only run processing if a local copy if needed. Local builds are required since github has a 100 mb file size limit.
#//*** If it's too annoying I should manually split the dataframe into multiple files, or see if pickle supports multi-part zip files.
#//********************************************************************************************************************************************

#//*** Runs when set to True
if False:
    
    #//*** Read the File
    df = pd.read_json("z_wk09_categorized-comments.jsonl", lines=True)

    #//*** Clean the Text.
    df['processed'] = mr_clean_text(df['txt'],{"lower": True, "newline": True, "html": True, "remove_empty" : False, "punctuation" : True})

    print(f"Articles of length with 0 characters: {len(df[ df['processed'].str.len() == 0 ])}")

    #//Remove Items with an Arbitrary length of 0

    print(f"Articles of length with 0 characters: {len(df[ df['processed'].str.len() == 0 ])}")
    print("Remove these articles")
    print(f"Article Count Before: {len(df)}")
    df = df[ df['processed'].str.len() > 0 ]
    print(f"Article Count After: {len(df)}")

    #//************************
    #//*** Tokenize the Text
    #//************************
    #//*** The custom function displays progress while it's working
    df['processed'] = tokenize_series(df['processed'],20,{"fast":True})

    #//************************
    #//*** Remove Stop Words
    #//************************
    #//*** The custom function displays progress while it's working
    df['processed'] = remove_stop_words(df['processed'])

    #//************************
    #//*** Apply Lematization
    #//************************
    df['lema_stem_tokens'] = apply_lemmatization(df['processed']) 

    print( f"Total Corpus Word Count: {df['lema_stem_tokens'].apply(lambda x: len(x)).sum()}" )
    #//*** Eliminate words with length of 0,1 or 2. This is an arbitrary value to help with feature reduction
    #df['tokens'] = df['tokens'].apply(lambda word_list : list(filter(lambda word : len(word) >= 3, word_list))) 
    df['lema_stem_tokens'] = df['lema_stem_tokens'].apply(lambda word_list : list(filter(lambda word : len(word) >= 3, word_list))) 

    #//*** Build Word Count
    df['num_wds'] = df['lema_stem_tokens'].apply(lambda x: len(x))

    print( f"Total Corpus Word Count: {df['lema_stem_tokens'].apply(lambda x: len(x)).sum()}" )

    #//************************
    #//*** Stem the lemma's
    #//***********************************************************************
    #//*** We are trying to reduce the feature set
    #//***********************************************************************
    df['lema_stem_tokens'] = apply_stemmer(df['lema_stem_tokens'])

    #//**********************************
    #//*** Apply Part of Speech Tagging
    #//**********************************
    df['pos_tag'] = apply_pos_tag(df['lema_stem_tokens'])
    
    #//**********************************
    #//*** Save the Dataframe to file
    #//**********************************
    pd.to_pickle(df,"z_wk09_categorized_comments_processed.zip")


In [4]:
#//**** Load the processed Text
df = pd.read_pickle("z_wk09_categorized_comments_processed.zip")

#//*** Convert categorical string to categorical int
#//*** Only run once to prevent iPython issues
if (df.dtypes['cat'] == object):
    cat_dict = dict(tuple(enumerate(df['cat'].unique())))
    #//*** Build sexcat Categorical column
    df['intcat'] = df['cat'].copy()
    
    #//*** replace values using the sex_dict dictionary
    for key,value in cat_dict.items():
        df['intcat'] = df['intcat'].replace(value,key)

#//*** Stingify the tokeninze Lema Stems
df['processed'] = df['lema_stem_tokens'].apply(lambda word_list: ' '.join(word_list)) 

In [5]:
print(df)

                cat                                                txt  \
0            sports  Barely better than Gabbert? He was significant...   
1            sports  Fuck the ducks and the Angels! But welcome to ...   
2            sports  Should have drafted more WRs.\n\n- Matt Millen...   
3            sports            [Done](https://i.imgur.com/2YZ90pm.jpg)   
4            sports                                      No!! NOO!!!!!   
...             ...                                                ...   
606471  video_games             No. It's probably only happened to you   
606472  video_games  I think most of the disappointment came from t...   
606473  video_games  dishonored 1/2 looked like arse, so what the h...   
606474  video_games                                          [removed]   
606475  video_games  I wish more games provided options like Rise o...   

                                                processed  \
0       bare better gabbert significantli better y

In [6]:
#conda install -c conda-forge transformers 

#from sklearn.externals import joblib







In [7]:
#//*****************************************************************************************
#//**** Lots of reference articles, keep them here for well, later reference
#//*****************************************************************************************
#https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
#https://analyticsindiamag.com/a-beginners-guide-to-scikit-learns-mlpclassifier/
#https://duckduckgo.com/?q=sklearn+cross_val_score+&t=ffab&ia=web
#https://towardsdatascience.com/sentiment-analysis-using-classification-e73da5b4159f
#//*****************************************************************************************

#//**** Load Sci Kit Learn Libraries. Importing here makes the code easier to export for future learnings
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [11]:


import time



#//*** X is Post Processed Data to evaluate
data_model_x = df['processed']


#//*** Categorical Integers are the target
data_model_y = df['intcat']


#//*** There 600,000 ish records. Training on more than 2% of the data, really brings the system to it's knees. In the interest of time and sci kit is kind slow.
#//*** I'm sticking with 2% training size
test_size=.98
#continuous scoring model
#scoring = 'r2_score'

# split the data randomly into test/train sets
x_train, x_test, y_train, y_test = train_test_split(data_model_x, data_model_y, test_size =test_size)

#categorical scoring model
#scoring = 'f1'

#print(f"Calculating Model: {len(x_train)} elements")

#//*** Initialize the Vectorizer, get all the features
tfidf = TfidfVectorizer()

print("First pass Vectorizing")
total_features = tfidf.fit_transform(x_train).shape[1]

print(total_features)

FEATURE_PERCENT = .1
SECOND_LAYER_SIZE = .5

N_FEATURES = int(total_features * FEATURE_PERCENT)

print(f"Re-Vectorizing: max_features={N_FEATURES} [{total_features}*{FEATURE_PERCENT}] Data Size: {len(x_train)}")

vectorizer = TfidfVectorizer(max_features=N_FEATURES)

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)


data_model_y = df['intcat']



#//*** Define the classifier model
classifier = Pipeline([
    #'norm', TextNormalizer(),
    #('tfidf',TfidfVectorizer()),
    #('ann',MLPRegressor(hidden_layer_sizes=[500,150], verbose=True))
    #('ann',MLPClassifier(hidden_layer_sizes=[500,150], verbose=True))
    ('ann',MLPClassifier(hidden_layer_sizes=[N_FEATURES,int(N_FEATURES*SECOND_LAYER_SIZE)], verbose=True))
])
#//*** Start Timing the process
start_time = time.time()

#//*** Train the Model
classifier.fit(x_train,y_train)

print(f"Modeling Time: {round(time.time()-start_time,2)}s")


First pass Vectorizing
14935
Re-Vectorizing: max_features=1493 [14935*0.1] Data Size: 12120
Iteration 1, loss = 0.65907100
Iteration 2, loss = 0.46079605
Iteration 3, loss = 0.31828674
Iteration 4, loss = 0.19361177
Iteration 5, loss = 0.13973386
Iteration 6, loss = 0.12476828
Iteration 7, loss = 0.11730836
Iteration 8, loss = 0.11576925
Iteration 9, loss = 0.11385224
Iteration 10, loss = 0.11113335
Iteration 11, loss = 0.10910524
Iteration 12, loss = 0.10729068
Iteration 13, loss = 0.10597469
Iteration 14, loss = 0.10587764
Iteration 15, loss = 0.10504469
Iteration 16, loss = 0.10428418
Iteration 17, loss = 0.10214203
Iteration 18, loss = 0.10297004
Iteration 19, loss = 0.10307815
Iteration 20, loss = 0.10121714
Iteration 21, loss = 0.10014710
Iteration 22, loss = 0.10051258
Iteration 23, loss = 0.10018764
Iteration 24, loss = 0.10086452
Iteration 25, loss = 0.09807039
Iteration 26, loss = 0.09982980
Iteration 27, loss = 0.09834192
Iteration 28, loss = 0.09760594
Iteration 29, loss = 

In [14]:
#//*** Generate Predictions
y_predicted = classifier.predict(x_test)

In [18]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score, confusion_matrix,classification_report
#//*** Score the Model

print(confusion_matrix(y_test, y_predicted))
print(classification_report(y_test, y_predicted))


[[ 63865   1607  77317]
 [  2426   7557  14584]
 [ 42105   6052 378375]]
              precision    recall  f1-score   support

           0       0.59      0.45      0.51    142789
           1       0.50      0.31      0.38     24567
           2       0.80      0.89      0.84    426532

    accuracy                           0.76    593888
   macro avg       0.63      0.55      0.58    593888
weighted avg       0.74      0.76      0.74    593888

